In [17]:
import pandas as pd
import csv 
df = pd.read_csv("rasa_events_processed.csv",delimiter=";",parse_dates =['timestamp']) # zastosowane 'relative paths',żeby program zadział nie tylko w wybranym folderze  
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
#df["sender_id"] = df["sender_id"].astype("category")
df = df.fillna("None")
print(df.head())

   Column1  id                         sender_id        type_name  \
0        0   1  5b4e6c442a364c9a8fdaf78bb61529a9           action   
1        1   2  5b4e6c442a364c9a8fdaf78bb61529a9  session_started   
2        2   3  5b4e6c442a364c9a8fdaf78bb61529a9           action   
3        3   4  5b4e6c442a364c9a8fdaf78bb61529a9             user   
4        4   5  5b4e6c442a364c9a8fdaf78bb61529a9           action   

            timestamp intent_name           action_name    text  \
0 2020-12-13 16:39:00        None  action_session_start    None   
1 2020-12-13 16:39:00        None                  None    None   
2 2020-12-13 16:39:00        None         action_listen    None   
3 2020-12-13 16:39:00       greet                  None  /greet   
4 2020-12-13 16:39:00        None           utter_greet    None   

              slot_name slot_value intent_confidence  
0  action_session_start       None              None  
1                  None       None              None  
2         action_

In [18]:
# Wskaźnik średnia liczba wiadomości na sesję w okresie czasu: 13-19.12.2020
def get_unique_users(df, start_date, end_date):
    truncated_df = df[(df["timestamp"] >= start_date) & (df["timestamp"] < end_date)]
    #agrs: 
    # df-dataFrame z pliku csv, 
    # start-date-typ date,data początkowa,
    # end_date-typ date, data końcowa
    # returns:
    #sessions, typ int, Liczba użytkowników od 2020-12-13 do 2020-12-19
    return truncated_df["sender_id"].nunique()

sessions = get_unique_users(df, "2020-12-13", "2020-12-19")
print(f"Liczba użytkowników: {sessions}")

def get_unique_texts(df, start_date, end_date):
    truncated_df = df[(df["timestamp"] >= start_date) & (df["timestamp"] < end_date)]
    #agrs: 
    # df-dataFrame z pliku csv, 
    # start-date-typ date, data początkowa,
    # end_date-typ date, data końcowa
    # returns:
    # messages, typ int, Liczba wiadomości od 2020-12-13 do 2020-12-19
    return truncated_df[truncated_df["type_name"] == "user"].shape[0]
# Żeby dostać liczbę wiadomości użytkowników w danym czasie trzeba wziąć wszystkie eventy gdzie 'type_name' jest równe 'user'
# truncated_df[truncated_df["type_name"] == "user"] - To jest filtr
# .shape[0] - To daję nam liczbę wierszy
messages = get_unique_texts(df, "2020-12-13", "2020-12-19")
print(f"Liczba wiadomości: {messages}")


def avg_messages_per_session(df, start_date, end_date):
    sessions = get_unique_users(df, start_date, end_date)
    messages = get_unique_texts(df, start_date, end_date)
    #agrs: 
    # df-dataFrame z pliku csv, 
    # start-date-typ date, data początkowa,
    # end_date-typ date, data końcowa
    # returns:
    # average_number_text: messages/sessions, typ float, Średnia liczba wiadomości na sesję w okreście czasu w okresie czasu od start_date do end_date.
    return messages/sessions
average_number_text = messages/sessions
start_date = "2020-12-13"
end_date = "2020-12-19"
average_messages = avg_messages_per_session(df, start_date, end_date)

print(f"Średnia liczba wiadomości na sesję w okreście czasu od {start_date} do {end_date}: {average_number_text}")

Liczba użytkowników: 14
Liczba wiadomości: 131
Średnia liczba wiadomości na sesję w okreście czasu od 2020-12-13 do 2020-12-19: 9.357142857142858


In [19]:
# Wskaźnik najpopularniejsze godziny korzystania
# Trzeba ograniczyć timestamp do 'czułości jaką chcemy osiągnąć" np. godziny, dni, dni tygodnia etc.
# W tej chwili wydobyto z timestampa godzinę, było to możliwe dzięki temu, że Timestamp jest już w typie danych datetime 

df["hour"] = df["timestamp"] # Kolumnę z samą godziną
df["hour"].value_counts() # Podaje wartośc

2021-01-14 17:47:00    43
2021-01-14 17:46:00    36
2021-01-15 19:31:00    31
2020-12-21 21:33:00    31
2020-12-13 17:36:00    27
                       ..
2020-12-17 14:26:00     4
2020-12-17 22:46:00     4
2020-12-17 14:45:00     4
2020-12-13 17:22:00     3
2020-12-13 16:47:00     2
Name: hour, Length: 119, dtype: int64

In [20]:
# Wskaźnik dotyczący liczby użytkowników w przedziale czasu(2020-12-14/202-12-19) 

def get_unique_users(df, start_date, end_date):
    truncated_df = df[(df["timestamp"] >= start_date) & (df["timestamp"] < end_date)]
    #agrs: 
    # df-dataFrame z pliku csv, 
    # start-date-typ date,data początkowa,
    # end_date-typ,date, data końcowa
    # returns:
    #nunique, typ int, Liczba użytkowników od 2020-12-13 do 2020-12-19
    return truncated_df["sender_id"].nunique()

nunique = get_unique_users(df, "2020-12-14", "2020-12-19")
print(f"Liczba użytkowników: {nunique}")

Liczba użytkowników: 13


In [21]:
# Wskaźnik dotyczący kategoryzowanie pytań oraz zliczania ich liczby
sorted_questions = df.groupby('intent_name').intent_name.count()# dokonano grupowania po intent_name a następnie zliczono dane pytania, liczba podana w typie int
print (sorted_questions)

intent_name
None                      986
adres                       5
affirm                      5
bufet                       2
continue                    2
deny                        7
dyplomowanie                2
dziekanat_explain_both      1
dziekanat_info              1
dziekanat_location          1
dziekanat_times             3
erasmus                     2
goodbye                     1
greet                      55
happy                       2
harmonogram                 1
help                        1
inform                     18
kierownictwo                1
kim_jestes                  1
konsultacje                25
kontakt                     8
legitymacja                 4
mail                        4
mapa                        5
nlu_fallback               19
oplaty_za_studia            3
out_of_scope                9
pokoj                       1
praktyki                    3
room                       19
sad                         1
sesja                       

In [22]:
# Wskaźnik dotyczący zliczania błędów 
errors = df[df["intent_name"] == "nlu_fallback"]
len(errors) # lub errors.shape[0] ten sam efekt, typ int


19

In [23]:
# Wskaźnik dotyczący wybierania zapytań, w których chatbot nie rozpoznał tekstu
df = pd.read_csv("rasa_events_processed.csv",delimiter=";",parse_dates =['timestamp'])

def get_not_recognized(df):
    user_texts = df[df["type_name"] == "user"]
    #agrs: 
    # df-dataFrame z pliku csv, 
    # returns:
    # user_texts,zwraca te zapytania, w których intent_name jest równy nlu_fallback
    return user_texts[user_texts["intent_name"] == "nlu_fallback"]

not_recognized = get_not_recognized(df)
not_recognized.head()

,Column1,id,sender_id,type_name,timestamp,intent_name,action_name,text,slot_name,slot_value,intent_confidence
207,207,190,5b4e6c442a364c9a8fdaf78bb61529a9,user,2020-12-13 17:36:00,nlu_fallback,NaN,BLA BLA,NaN,NaN,0.672338
222,222,208,5b4e6c442a364c9a8fdaf78bb61529a9,user,2020-12-13 17:36:00,nlu_fallback,NaN,BLA BLA,NaN,NaN,0.672338
229,229,217,5b4e6c442a364c9a8fdaf78bb61529a9,user,2020-12-13 17:37:00,nlu_fallback,NaN,BLA BLA,NaN,NaN,0.672338
246,246,231,5b4e6c442a364c9a8fdaf78bb61529a9,user,2020-12-13 17:38:00,nlu_fallback,NaN,BLA BLA,NaN,NaN,0.672338
283,283,268,5b4e6c442a364c9a8fdaf78bb61529a9,user,2020-12-13 18:11:00,nlu_fallback,NaN,do domu ziomek,NaN,NaN,0.655446


In [24]:
# Wskaźnik dodatkowy dotyczący oceny rozmowy
# przykładowo zrobiono na podstawie wartości intent_name, w których chatbot nie rozpoznał tesktu co
# może wpłynąć na niską ocenę chatbota przez użytkownika, ocena w gwiazdkach od 1-5;
# W planach jest posiadanie oceny użytkownika, dzięki czemu będzie można wydobyć intent np. ocena_rozmowy i wartość jaką wybrał użytkownik
df = pd.read_csv("rasa_events_processed.csv",delimiter=";",parse_dates =['timestamp'])
def stars(row):
    if row.intent_name == "nlu_fallback":
        return 1
    else:
        return 5
#agrs: 
# row-wiersze, 
# returns:
# 1 lub 5, typ int, w zależnosći(warunek if) od tego jeśli w wierszu intent_name będzie równe nlu_fallback(brak rozpoznania tekstu) to wtedy zwracana jest 1 a jeśli nie jest równy w wierszu intent_name nlu_fallback to 5
star_ratings = df.apply(stars, axis='columns')
print(star_ratings)

0       5
1       5
2       5
3       5
4       5
       ..
1232    5
1233    5
1234    5
1235    5
1236    5
Length: 1237, dtype: int64
